# Diana Uribe

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
total_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'EDITORIAL', 'SUCURSAL', 'DESC_SUCURSAL', 'COD_BARRAS_LIBRO', 'NUM_TITULO', 'SIG_TOPOGRAFICA',
    'DEWEY', 'II_SUMARIO', 'I_SUMARIO', 'EDICION', 'LOCALIZACION_EJEMPLAR', 'DESC_LOCALIZACION_EJEMPLAR', 'POLITICA',
    'DESC_POLITICA_EJEMPLAR', 'ISBN', 'FECHA_REGISTRO_EJEMPLAR', 'FECHA_ADICION_EJEMPLAR', 'FECHA_PUB',
    'ESTADO_EJEMPLAR', 'PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION', 'NIVEL_EDUCACION', 'AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO', 'FECHA_REAL_DEVOLUCION',
]

In [3]:
used_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'SUCURSAL', 'COD_BARRAS_LIBRO', 'DEWEY', 'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB','PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION','NIVEL_EDUCACION','AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO','SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO','FECHA_REAL_DEVOLUCION'
]

# Préstamos

In [4]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 19.1 s, sys: 926 ms, total: 20.1 s
Wall time: 20.1 s


In [5]:
prestamos_df.shape

(3054360, 20)

In [6]:
prestamos_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
0,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429527,CTA,22/01/2015,04/02/2015,2015-01-22,2015-02-04
1,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88416381,CTA,31/03/2015,07/04/2015,2015-03-31,2015-04-07
2,Quisiera tener ...,"Zoboli, Giovanna.",ARM,2284458,853,Infantil,2010,mx,,,,ARM,ARM,ARM,88381353,CTA,07/11/2015,17/11/2015,2015-11-07,2015-11-17
3,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,Adm. de empresas,ARM,ARM,ARM,88007424,CEM,16/01/2015,16/01/2015,2015-01-16,2015-01-16
4,La fantástica leyenda de: la princesa y el dragón,"Alins, Sonia.",ARM,2288896,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429529,CTA,21/01/2015,28/01/2015,2015-01-21,2015-01-28


# Diana Uribe

In [10]:
prestamos_df[(prestamos_df.AUTOR.str.contains('Diana')) & prestamos_df.AUTOR.str.contains('Uribe')].drop_duplicates(['TITULO'])

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
9109,África : nuestra tercera raiz,"Uribe, Diana.",BLAA,2602475,960,Depósito D2 900-999,2014,ck,Universidad,Profesional,Pedagogía,CASAGO,CASAGO,CASAGO,88452887,CTF,24/09/2015,19/10/2015,2015-09-24,2015-10-19
35378,Alexandros,"Uribe, Diana.",BLAA,2211396,Registro sonoro en sala/pres.,Audiovisuales,2002,ck,Independiente,Postgrado,Artes plásticas,ESPE,CASAGO,CASAGO,88452997,CTF,26/04/2015,04/05/2015,2015-04-26,2015-05-04
40284,La historia en los viajes : los viajes en la h...,"Uribe, Diana.",BLAA,2348924,910,Depósito D2 900-999,2011,ck,Empleado,Profesional,Ing. industrial,BLAA,BLAA,BLAA,88440091,CTB,07/03/2015,06/04/2015,2015-03-07,2015-04-06
44065,Ataque a Irak,"Uribe, Diana.",BLAA,1794854,Registro sonoro en sala/pres.,Audiovisuales,2003,ck,Independiente,Postgrado,Artes plásticas,ESPE,CASAGO,CASAGO,88452997,CTF,05/06/2015,12/06/2015,2015-06-05,2015-06-12
48789,Brazil amazónico,"Uribe, Diana.",BLAA,1769691,Registro sonoro en sala/pres.,Audiovisuales,2003,ck,Independiente,Postgrado,Artes plásticas,CASAGO,CASAGO,CASAGO,88452997,CTF,17/06/2015,23/06/2015,2015-06-17,2015-06-23
49033,Brasil actual,"Uribe, Diana.",BLAA,1805145,Registro sonoro en sala/pres.,Audiovisuales,2003,ck,Independiente,Postgrado,Artes plásticas,ESPE,CASAGO,CASAGO,88452997,CTF,05/06/2015,12/06/2015,2015-06-05,2015-06-12
49082,Brasil introducción,"Uribe, Diana.",BLAA,1769702,Registro sonoro en sala/pres.,Audiovisuales,2003,ck,Independiente,Postgrado,Artes plásticas,CASAGO,CASAGO,CASAGO,88452997,CTF,17/06/2015,23/06/2015,2015-06-17,2015-06-23
50554,Califato de Córdoba al Andaluz,"Uribe, Diana.",BLAA,1796757,Registro sonoro en sala/pres.,Audiovisuales,2003,ck,Docente,Profesional,Matemáticas,ESPE,BLAA,BLAA,88347871,CTC,24/04/2015,02/05/2015,2015-04-24,2015-05-02
54602,China I,"Uribe, Diana.",BLAA,1777437,Registro sonoro en sala/pres.,Audiovisuales,2002,ck,Independiente,Postgrado,Artes plásticas,CASAGO,CASAGO,CASAGO,88452997,CTF,09/07/2015,16/07/2015,2015-07-09,2015-07-16
54603,China II,"Uribe, Diana.",BLAA,1777436,Registro sonoro en sala/pres.,Audiovisuales,2002,ck,Independiente,Postgrado,Artes plásticas,CASAGO,CASAGO,CASAGO,88452997,CTF,09/07/2015,16/07/2015,2015-07-09,2015-07-16
